# Special Methods on 'Object'
- methods of the form ```__...__``` play special roles in implementing classes
- the first argument of ANY method is always 'self'
- the class author defines these methods, but they are not normally called directly
    - some of the methods are invoked by operators
        - you can define what '+', '*', etc, means for an object you define
        - C++ allows this, Java doesn't
    - other methods are invoked by well known functions

# Basic
- ```__init__```  - called at object creation time. used to initialize object state
- ```__len__```  - 'len' function will call this method
- ```__bool__```  - 'bool' function will call this method
- ```__str__, __repr__``` - controls how object prints
- ```__contains__``` - used by 'in' operator
- ```__call__``` - call an object like a function call

In [1]:
class P:
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        # 'friendly' string representation of object
        return('str method: ' + self.name)
    
    def __repr__(self):
        # 'detailed' string representation of object
        return('repr method: ' + self.name)
    
    def __len__(self):
        return len(self.name)
    
    def __bool__(self):
        # long name => true
        # short name => false
        return len(self.name) > 5
    
    def __call__(self, arg):
        return arg + 10
    
    def __contains__(self, arg):
        return arg == self.name

p = P('jack')

# top level uses __repr__
p

repr method: jack

In [2]:
# print uses __str__

print(p)

str method: jack


In [3]:
# note - 'in' operator will search for substrings

'foo' in 'bazfoobar'

True

In [4]:
['jack' in p, 'jill' in p]

[True, False]

In [5]:
bool(p)

False

In [6]:
bool(P('jackson'))

True

In [7]:
len(P('jackson'))

7

In [8]:
# call an object like a function

p = P('afd')
p(34)

44

In [9]:
# 'repr' and 'str' are just methods, so an object can print 
# differently at any time, depending on whatever
# very common to display some kind of status summary for the object

import random

class P2:
    def __repr__(self):
        return(random.choice(['ready', 'waiting', 'running', 'finished']))
p2 = P2()
for j in range(10):
    print(p2)

finished
waiting
running
waiting
running
finished
running
waiting
finished
finished


# Arithmetic
- x is an instance of class X
- y is an instance of class Y

- demonstrate ```__add__```, ```__radd__```, ```__iadd__``` below
- likewise for ```__mul__, __rmul__, __imul__ ```

In [10]:
class X:
    def __init__(self, num):
        self.xnum = num

    def __str__(self):
        return repr(self)
    
    def __repr__(self):
        return 'X({})'.format(self.xnum)

    def addInternal(self, n):
        if isinstance(n, X):
            return self.xnum + n.xnum
        if isinstance(n, Y):
            return self.xnum + n.ynum
        # either int OR float
        if isinstance(n, (int, float)):
            return self.xnum + n
    
    def __add__(self, y):
        return self.addInternal(y)
    
    # increment x
    def __iadd__(self, n):
        self.xnum = self.addInternal(n)
        # must return self
        return self
    
    # x is in the right side
    def __radd__(self, n):
        return self.addInternal(n)


class Y:
    # does not have __add__, __iadd__, or __radd__ methods
    def __init__(self, num):
        self.ynum = num

    def __str__(self):
        return('Y({})'.format(self.ynum))
    
    def __repr__(self):
        return str(self)




In [11]:
# run x __add__ method

x = X(23)
y = Y(10)
[x, y, x + y, x + 4, x + 5.5]

[X(23), Y(10), 33, 27, 28.5]

In [12]:
# run x __iadd__ method

x += 23
x

X(46)

In [13]:
# run x __radd__ method
# x is on the right side

y + x

56

# Comparision
- ```__lt__, __gt__, __le__, __ge__, __eq__, __ne__``` 
- tedious - many methods to define
- easier way is to use functools.total_ordering - only need to define two
- called by operators like '==' and '<'

In [14]:
from functools import total_ordering

# total_ordering is a 'decorator'
# it will 'write' the other four predicates 'by magic'

@total_ordering
class Student:
    def __init__(self, first, last):
        # don't let case confuse sort
        self.firstname = first.lower()
        self.lastname = last.lower()
        
    def __eq__(self, other):
        return ((self.lastname, self.firstname) ==
                (other.lastname, other.firstname))
    
    def __lt__(self, other):
        return ((self.lastname, self.firstname) <
                (other.lastname, other.firstname))

s1 = Student('joe', 'college')
s2 = Student('jack', 'junior')

In [15]:
s1 is s2

False

In [16]:
s1 == s1

True

In [17]:
s1 == s2

False

In [18]:
s1 < s2

True

In [19]:
s1 > s2

False

# Iteration
- ```__iter__``` - return an 'iterable' for this object
    - 'iter' function calls this method
- ```__next__``` - call on an iterable to get the next element in the sequence. raises 'StopIteration' error when sequence is exhausted
    - 'next' function calls this method

In [20]:
# using the actual methods...

x = [32,4]
it = x.__iter__()
print(it)
print(it.__next__())
print(it.__next__())
it.__next__()


32
4


StopIteration: 

In [21]:
# using the functions - same thing, but nicer looking

x = [32,4]
it = iter(x)
print(it)
print(next(it))
print(next(it))
next(it)


32
4


StopIteration: 

# collection element access and slices 
- ```__getitem__``` - get element or slice
- ```__setitem__``` - set element or slice

In [22]:
    def __getitem__(self, index):
        # do different things depending on type of 'index'
        if isinstance(index, int):
            # if asked for a single term, p[n], index will
            # be an int
            pass
        if isinstance(index, slice):
            # if asked for a slice, p[n:m], index will be
            # a 'slice' object
            pass
        pass

# Context Management (with statement)
- ```__enter__``` - acquire resource
- ```__exit__``` - release resource

In [23]:
with open('/tmp/path') as fd:
    fd.read()


FileNotFoundError: [Errno 2] No such file or directory: '/tmp/path'

In [24]:
class P:
    def __enter__(self):
        print('enter')
        return 23
    
    def __exit__(self, *pos):
        # on an error, pos will have info
        print(pos)
        print('exit')

with P() as p:
    print(p)


enter
23
(None, None, None)
exit


# Hashing
- ```__hash__``` - should only be defined for immutable objects. the hash of a mutable object could change, making it a poor key
- can turn it off this way

```
class foo:
    __hash__ = None
...
```

In [25]:
# a dictionary won't allow a mutable as a key

k = [1,3,4]
d = {}
d[k] = 2

TypeError: unhashable type: 'list'

# Managed attributes
- sometimes you want to run code when a object attribue is accessed or set
- one way to do this is with decorators
- advanced technique

In [26]:
import random

# inherit from object
class RandomService:
    def __init__(self):
        self.counter = 0 
        
    @property
    #  this runs on a "get"
    # inc counter, rtn random
    def ran(self):
        self.counter += 1
        return(random.random())
    
    @ran.setter
    # this runs on a "set"
    # set seed, reset counter
    def ran(self, val):
        self.counter = 0
        if not isinstance(val, float):
            raise TypeError('Expected a float')
        random.seed(val)
    
rs = RandomService()    
 

In [27]:
[rs.ran for j in range(5)]


[0.8614990813518001,
 0.49807244568371833,
 0.9093319151497646,
 0.7783122652226409,
 0.24793050197402144]

In [28]:
# counter recorded 5 invocations

rs.counter

5

In [29]:
# wants a float
rs.ran = 'asfd'

TypeError: Expected a float

In [30]:
# set seed, reset counter
rs.ran = 1.0
rs.counter

0

In [31]:
[rs.ran for j in range(5)]

[0.13436424411240122,
 0.8474337369372327,
 0.763774618976614,
 0.2550690257394217,
 0.49543508709194095]

In [32]:
# reset seed, same random numbers
rs.ran = 1.0
[rs.ran for j in range(5)]

[0.13436424411240122,
 0.8474337369372327,
 0.763774618976614,
 0.2550690257394217,
 0.49543508709194095]